### Oppgave 2: Finne landene med høyeste og laveste BNP pr. capita
Vi tar utgangspunkt i datasettet om innbyggerantall registrert i 2020. Det finnes foreløpig bare ett datasett med oversikt over landenes BNP.

Fremgangsmåte:
1. Finne eksempeldataene på dataplattformen
2. Lese inn eksempeldataene til pyspark.sql.DataFrame
3. Fjerne missing eller null verdier for innbyggerantall og BNP
4. Fjerner dubletter fra datasettene
5. Kobler datasettene
6. Lager ny variabel BNP pr. capita
7. Finner høyeste og lavest BNP pr. capita

#### Laster inn bibliotek:

In [ ]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, min, max
import pyspark.sql.functions as F

#### 2.1 Finne datasettene

In [ ]:
# finne datasettene om innbyggerantall og bnp
spark.read.path("/felles/veiledning/pyspark/eksempler*").show(10, False)

#### 2.2 Lese inn dataene
Kopier filstiene fra resultatet (output) fra forrige celle:
- /felles/veiledning/pyspark/eksempler/bnp
- /felles/veiledning/pyspark/eksempler/innbyggerantall/2020

In [ ]:
# lese inn datasettet om landenes bnp
df_bnp = spark.read.path("/felles/veiledning/pyspark/eksempler/bnp")

# lese inn datasettet om landenes innbyggerantall
df_innbyggerantall_2020 = spark.read.path("/felles/veiledning/pyspark/eksempler/innbyggerantall/2020")

# skriver ut dataene
print("*********************************** BNP ***********************************")
df_bnp.show(50,False)
print("*********************************** INNBYGGERANTALL 2020 ***********************************")
df_innbyggerantall_2020.show(50,False)

#### 2.3 Fjerne observasjoner som er 0, missing eller negative

In [ ]:
# filtrerer de observasjonenen hvor Innbyggerantall>0
df_innbyggerantall_2020 = df_innbyggerantall_2020.filter(df_innbyggerantall_2020.Innbyggerantall>0)
df_innbyggerantall_2020.show(50, False)

In [ ]:
# filtrerer de observasjonenen hvor BNP>0
df_bnp = df_bnp.filter(df_bnp.BNP>0)
df_bnp.show(50, False)

#### 2.4 Fjerne dubletter

In [ ]:
# finne og eventuelt fjerne dubletter på datasettet innbyggerantall

# sorterer etter land og innbyggerantall og lister ut den sorterte dataframen
print("*********************************** INNBYGGERANTALL SORTERT ETTER LAND OG INNBYGGERANTALL ***********************************")

df_innbyggerantall_2020_sort = df_innbyggerantall_2020.orderBy(['Land','Innbyggerantall'], ascending=[1, 0])
df_innbyggerantall_2020_sort.show(50,False)

# tar ut oversikt over de radene med likt "land" som forekommer flere enn en gang i dataframen, og sånn sett utgjør den samlede mengden av duplikater
df_land_dub_alle  = df_innbyggerantall_2020_sort.join(df_innbyggerantall_2020_sort.groupBy('Land')\
          .count().where('count = 1').drop('count'),
          on=['Land'],
          how='left_anti')

print("*********************************** OVERSIKT OVER DUPLIKATER ***********************************")
df_land_dub_alle.show()  

print("*********************************** INNBYGGERANTALL UTEN DUBLIKATER. HØYESTE INNBYGGERANTALL ER BEHOLDT ***********************************")
df_innbyggerantall_udub= df_innbyggerantall_2020_sort.drop_duplicates(subset=['Land'])
df_innbyggerantall_udub.show(50,False)


In [ ]:
# finne og eventuelt fjerne dubletter på datasettet BNP

# sorterer etter land og innbyggerantall og lister ut den sorterte dataframen
print("*********************************** BNP SORTERT ETTER LAND OG STØRRELSE PÅ BNP ***********************************")

df_bnp_sort = df_bnp.orderBy(['Land','BNP'])
df_bnp_sort.show(50,False)

# tar ut oversikt over de radene med likt "land" som forekommer flere enn en gang i dataframen, og sånn sett utgjør den samlede mengden av duplikater
df_bnp_dub_alle  = df_bnp_sort.join(df_bnp_sort.groupBy('Land')\
          .count().where('count = 1').drop('count'),
          on=['Land'],
          how='left_anti')

print("*********************************** OVERSIKT OVER DUPLIKATER ***********************************")
df_bnp_dub_alle.show()  

#### 2.5 Koble datasett

In [ ]:
# koble datasettene med bruk av koblingsmetoden 'inner' (vi ønsker bare å ha med land som har BNP og Innbyggerantall)

df_bnp_innbyggerantall = df_bnp.join(df_innbyggerantall_udub, 'Land', 'inner')

print("*********************************** SAMMENKOBLET DATASETT ***********************************")
df_bnp_innbyggerantall.show(50,False)

#### 2.6 Lage en ny variabel og fjerner kolonner

In [ ]:
# lage ny variabel (BNP/innbyggerantall)

df_bnp_innbyggerantall = df_bnp_innbyggerantall.withColumn('BNP per capita',(df_bnp_innbyggerantall['BNP']/df_bnp_innbyggerantall['Innbyggerantall']).cast(IntegerType()))

# fjerner kolonnene Landkode, År og Kilde
df_bnp_innbyggerantall = df_bnp_innbyggerantall.drop("Landkode", "År", "Kilde")

print("*********************************** SAMMENKOBLET DATASETT MED NY VARIABEL ***********************************")
df_bnp_innbyggerantall.show(50,False)

#### 2.7 Finne min og maks (og gjennomsnitt)

In [ ]:
# finne min, maks og gjennomsnitt av den nye variabelen

result = df_bnp_innbyggerantall.select([(mean("BNP per capita")).cast(IntegerType()), min("BNP per capita"), max("BNP per capita")])

print("*********************************** Minimum, maksimum og gjennomsnitt av BNP per capita ***********************************")
result.show()